### Importing necessary packages :

In [6]:
import os
import pandas as pd
import platform
import shutil
import re

### Checking Operating System and configuring project's paths accordingly :

In [7]:
rootpath_analysis = ""
rootpath_data = ""
system = platform.system()
print(system, "plateform detected.")

try:
    if system == "Darwin":
        raise OSError
    elif system == "Linux":
        rootpath_analysis = r"/network/lustre/iss02/charpier/analyses/priam/EpiRest_Spike"
        rootpath_data = r"/network/lustre/iss02/epimicro/patients/raw"
        restpath_data = r"/network/lustre/iss02/epimicro/patients/protocols/epiRest"
        source_path   = r"/network/lustre/iss02/charpier/analyses/vn_onset/EpiCode/projects/epinjector"
    elif system == "Windows":
        rootpath_analysis = r"\\l2export\iss02.charpier\priam\EpiRest_Spike"
        rootpath_data = r"\\l2export\iss02.epimicro\patients\raw"
        restpath_data = r"\\l2export\iss02.epimicro\patients\protocols\epiRest"
        source_path   = r"\\l2export\iss02.charpier\analyses\vn_onset\EpiCode\projects\epinjector"
    else:
        raise OSError
except OSError:
    print(system, "platform not supported. Exiting...")

output_path = os.path.join(rootpath_analysis, "data", "patients")
image_path = os.path.join(rootpath_analysis, "data", "images")
templates_path = os.path.join(source_path, "src", "templates")
    
print(r"Raw Patients Path: ", rootpath_data)
print(r"Resting Patients Path: ", restpath_data)
print(r"Images Path: ", image_path)
print(r"Ouput Path: ", output_path)
print(r"Source Path: ", source_path)

Windows plateform detected.
Raw Patients Path:  \\l2export\iss02.epimicro\patients\raw
Resting Patients Path:  \\l2export\iss02.epimicro\patients\protocols\epiRest
Images Path:  \\l2export\iss02.charpier\priam\EpiRest_Spike\data\images
Ouput Path:  \\l2export\iss02.charpier\priam\EpiRest_Spike\data\patients
Source Path:  \\l2export\iss02.charpier\analyses\vn_onset\EpiCode\projects\epinjector


### Checking for matching files in root and rest :

In [8]:
root_path_list = []
rest_path_list = []
for path in os.listdir(rootpath_data):
    if path.startswith("pat"):
        root_path_list.append(path)
for path in os.listdir(restpath_data):
    if path.startswith("pat"):
        rest_path_list.append(path)

In [9]:
common_list = []
for common in set(root_path_list).intersection(set(rest_path_list)):
    if common.startswith("pat"):
        common_list.append(common)
try:
    if not len(list(set(rest_path_list) - set(common_list))):
        print("All rest patients linked to raw patients.")
    else:
        raise OSError
except OSError:
    print(list(set(rest_path_list) - set(common_list)))
    print("Rest and raw patients mismatch detected. Exiting...")  
print("Raw patients count:", len(root_path_list))
print("Rest patients count:", len(rest_path_list))

All rest patients linked to raw patients.
Raw patients count: 96
Rest patients count: 22


In [29]:
def write_global_config_file(templates_path, output_path, filename):
    if not os.path.exists(os.path.join(output_path, filename)):
        # create a global config file
        with open(os.path.join(templates_path, filename) ,'r') as template, \
             open(os.path.join(output_path, "global.cfg"),'w') as outputfile:
            outputfile.write("%% GLOBAL PARAMETERS\n")
            for line in template:
                if (not line.startswith("%")):
                    outputfile.write(line)
            outputfile.write("\n")
        template.close()
        outputfile.close()

In [30]:
def write_patient_config_file(templates_path, output_path, filename,
                              patient):
    if not os.path.exists(os.path.join(output_path, patient) + ".cfg"):
        # create a patient config file
        with open(os.path.join(templates_path, filename) ,'r') as template, \
             open(os.path.join(output_path, patient + ".cfg"),'w') as outputfile:
            outputfile.write("\n%% PATIENT PARAMETERS - " + patient + "\n")
            for line in template:
                if (not line.startswith("%")):
                    outputfile.write(line)
            template.close()
            outputfile.close()

In [31]:
def write_session_config_file(templates_path, output_path, filename,
                              patient, session):
    if not os.path.exists(os.path.join(output_path, session) + ".cfg"):
        # create a patient config file
        with open(os.path.join(templates_path, filename) ,'r') as template, \
             open(os.path.join(output_path, session + ".cfg") ,'w') as outputfile:
            outputfile.write("%SESSION PARAMETERS - " + session + "\n")
            for line in template:
                if (not line.startswith("%")):
                    outputfile.write(line)
            outputfile.write("\n")
        template.close()
        outputfile.close()

### Generating Output Folder :

In [32]:
os.listdir(rootpath_analysis)

['data', 'Main EpiRest Database.xlsx']

In [33]:
# Creating all non-existing output folders :
if not os.path.exists(output_path):
    os.makedirs(output_path)
if not os.path.exists(image_path):
    os.makedirs(image_path)

### Generating Global Config File :

In [34]:
write_global_config_file(templates_path, output_path, "global_template.cfg")

### Generating All Patients & Sessions Folders and Config Files :

In [35]:
for patient in rest_path_list:
    # Joining output path and patients path components :
    patient_path = os.path.join(output_path, patient)
    # Creating non-existing patient folder :
    if not os.path.exists(patient_path):
        os.makedirs(patient_path)
    # Generating patient config file
    write_patient_config_file(templates_path, patient_path,
                              "patient_template.cfg",
                              patient)
    for session in os.listdir(os.path.join(restpath_data, patient)):
        # Joining output path, patient path, and session path components :
        session_path = os.path.join(output_path, patient, session)
        if session.startswith("ses"):
            # Creating non-existing session folder :
            if not os.path.exists(session_path):
                os.makedirs(session_path)
            write_session_config_file(templates_path, session_path,
                                      "session_template.cfg",
                                      patient, session)

### Generating Matlab Parameter File

In [13]:
def generate_new_params_file(filename):
    final_params = open(filename, "w")
    global_params = open(os.path.join(output_path, "global") + ".cfg", 'r',)
    for line in global_params:
        final_params.write(line)
    final_params.write("%========================================================\n")
    global_params.close()
    session_id = 1
    for patient in rest_path_list:
        for session in os.listdir(os.path.join(output_path, patient)):
            if session.startswith("ses"):
                patient_path = os.path.join(output_path, patient)
                patient_params = open(os.path.join(patient_path, patient) + ".cfg", 'r')
                for line in patient_params:
                    new_line = re.sub('config{([0-9]+)}',
                                      'config{'+ str(session_id) +'}', line)
                    final_params.write(new_line)
                patient_params.close()
                session_output_path = os.path.join(output_path, patient, session)
                session_params = open(os.path.join(session_output_path, session) + ".cfg", 'r')
                for line in session_params:
                    new_line = re.sub('config{([0-9]+)}',
                                      'config{'+ str(session_id) +'}', line)
                    final_params.write(new_line)
                session_params.close()
                session_id += 1
        final_params.write("\n%--------------------------------------------------------\n")
    final_params.close()

In [14]:
generate_new_params_file("EpirestSpike_setparams.m")

FileNotFoundError: [Errno 2] No such file or directory: '\\\\l2export\\iss02.charpier\\priam\\EpiRest_Spike\\data\\patients\\pat_02680_1158\\ses-06\\ses-06.cfg'

In [1]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.EpirestSpike_prepare_SC(nargout=0)
eng.quit()